In [1]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
from Optimiser.config import get_config, normalize_action, denormalize_action
import argparse

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.9985486851366765

In [5]:
import tempfile
import os
BASE_DIR = "/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result"
TMP_SUBDIR = os.path.join(BASE_DIR, "tmp")

# make sure it exists once at startup:
os.makedirs(TMP_SUBDIR, exist_ok=True)
def haaqi_reward_muti_fn(solution: np.ndarray, is_normalised=True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
    print(f'solution:{solution}')
    thre, ratio, attack, release = solution

    # Create a unique temp‐file name
    fd, degradated_filename = tempfile.mkstemp(prefix="dynC_", suffix=".wav")
    os.close(fd)  # we’ll let your compressor write to that path

    try:
        gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(
            outputfilename=degradated_filename,
            threshold=thre,
            ratio=ratio,
            attack=attack,
            release=release)
        gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio, 64)
        score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
    finally:
        # clean up
        try:
            os.remove(degradated_filename)
        except OSError:
            pass

    return round(1 - score, 3)

In [8]:
Noise_Generator_MP3.EraseTheMp3Mixing()

All files and folders in '/home/codecrack/CodecBreakerwithRL/AudioEX//Mixing_Result' have been removed.


In [ ]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

In [ ]:
trainner = CRLTrain(sub_episode_length=3, sub_episode_num_single_batch=2, env_num=4)
trainner.set_environments(haaqi_reward_muti_fn)
trainner.train(update_num=3, eval_intv=1)

### GA Setting

In [ ]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [ ]:
import multiprocessing
n_cores = multiprocessing.cpu_count()

In [ ]:
n_cores

In [ ]:
GA_Opt = GeneticOptimiser()
GA_Opt.ga_init_env()
GA_Opt.set_fitnessfun(haaqi_reward_muti_fn)

In [ ]:
%%timeit -n 1 -r 1
GA_Opt.run(num_generations = 3,sol_per_pop = 5, parallel_processing=4)

### Mento Carlos

In [6]:
import Optimiser.mc_opt
from Optimiser.mc_opt_multi import MonteCarloMultiOptimiser

In [7]:
%%timeit -n 1 -r 1
mc = MonteCarloMultiOptimiser(reward_fn = haaqi_reward_muti_fn)
per_step_bests = mc.optimise_by_steps(n_steps=5, sampling_per_step=3)

Step 1: sampling 3 candidates
solution:[-13.69   9.42  16.5  101.1 ]solution:[-10.89   3.43   1.78 106.61]solution:[ -5.6    9.21  12.53 391.8 ]


solution:[ -4.28   1.3   14.86 170.26]solution:[-29.15   2.12  13.74 358.88]solution:[ -4.1    5.87   6.69 269.07]


Step 1: received 3 rewards
 Step 1/5 → best R = 0.082 with solution [-13.69   9.42  16.5  101.1 ]
Step 2: sampling 3 candidates
solution:[-11.54   4.45  19.95 492.33]solution:[-25.95   7.49  10.98 224.1 ]solution:[ -9.43   6.85  14.08 255.57]


Step 2: received 3 rewards
 Step 2/5 → best R = 0.149 with solution [-29.15   2.12  13.74 358.88]
Step 3: sampling 3 candidates
solution:[-12.85   3.9   12.29 235.16]solution:[-15.42   9.01  18.75 243.12]solution:[-18.25   9.01   5.32 349.27]


Step 3: received 3 rewards
 Step 3/5 → best R = 0.172 with solution [-25.95   7.49  10.98 224.1 ]
Step 4: sampling 3 candidates
solution:[ -3.71   1.53   7.39 160.11]solution:[-27.48   8.49  15.95 195.75]
solution:[-16.49   8.17   5.38 120.81]

S

In [ ]:
import Optimiser.mc_opt
from Optimiser.mc_opt import MonteCarloOptimiser

In [ ]:
%%timeit -n 1 -r 1
mc = MonteCarloOptimiser(reward_fn = haaqi_reward_muti_fn)
per_step_bests = mc.optimise_by_steps(n_steps=5, sampling_per_step=3)